# Scikit-learn 내장 데이터셋: load_digits 분석 및 분류 모델링

이 노트북은 Scikit-learn에서 제공하는 손글씨 숫자(Digits) 데이터셋을 활용하여 데이터 분석(EDA), 전처리, 특성 엔지니어링, 그리고 6가지 머신러닝 모델을 통한 학습 및 앙상블 모델 구축 과정을 담고 있습니다.

## 데이터셋 설명
- **데이터셋 이름**: Optical Recognition of Handwritten Digits Data (손글씨 숫자 데이터)
- **데이터 개수**: 1,797개
- **특성(Features)**: 64개 (8x8 픽셀 이미지의 그레이스케일 값, 0~16 범위)
- **타겟(Target)**: 0부터 9까지의 숫자 클래스
- **주요 목표**: 8x8 픽셀 데이터를 입력받아 해당 숫자가 무엇인지 분류하는 다중 클래스 분류 문제 해결

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 모델 라이브러리
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

## 1. 데이터 로드 및 탐색적 데이터 분석 (EDA)

In [ ]:
# 데이터 로드
digits = load_digits()
X = digits.data
y = digits.target

print(f"데이터셋 크기: {X.shape}")
print(f"타겟 클래스: {np.unique(y)}")

# 데이터프레임 변환 (확인용)
df = pd.DataFrame(X, columns=[f'pixel_{i}' for i in range(X.shape[1])])
df['target'] = y
df.head()

### 1.1 데이터 시각화
실제 8x8 이미지가 어떻게 생겼는지 확인해 봅니다.

In [ ]:
plt.figure(figsize=(12, 5))
for index, (image, label) in enumerate(zip(digits.images[:10], digits.target[:10])):
    plt.subplot(2, 5, index + 1)
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title(f'Target: {label}')
    plt.axis('off')
plt.tight_layout()
plt.show()

### 1.2 클래스 분포 확인

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(x=y)
plt.title("Distribution of Target Classes")
plt.xlabel("Digit")
plt.ylabel("Count")
plt.show()

## 2. 데이터 전처리 및 특성 엔지니어링

In [ ]:
# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 스케일링 (픽셀 값이 0~16으로 제한적이지만, 일부 모델의 수렴을 위해 표준화 수행)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

## 3. 모델링 (6가지 모델 사용)

In [ ]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=10000),
    "SVC": SVC(probability=True),
    "RandomForest": RandomForestClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, preds)
    results[name] = acc
    print(f"{name} Accuracy: {acc:.4f}")

## 4. 상위 4개 모델 선정 및 앙상블 모델 구축
성능이 좋은 4가지 모델을 추려서 하이퍼 파라미터 튜닝을 진행하고 앙상블(Soft Voting) 모델을 만듭니다.

In [ ]:
# 성능 기준 내림차순 정렬
sorted_models = sorted(results.items(), key=lambda x: x[1], reverse=True)
top_4_names = [m[0] for m in sorted_models[:4]]
print(f"Top 4 Models: {top_4_names}")

# 간단한 튜닝 (GridSearchCV 예시 - 시간 관계상 일부 파라미터만 시연)
tuned_models = []

for name in top_4_names:
    if name == 'SVC':
        param_grid = {'C': [1, 10], 'gamma': ['scale', 0.01]}
        grid = GridSearchCV(SVC(probability=True), param_grid, cv=3).fit(X_train_scaled, y_train)
        tuned_models.append(('svc', grid.best_estimator_))
    elif name == 'LogisticRegression':
        param_grid = {'C': [0.1, 1.0, 10.0]}
        grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=3).fit(X_train_scaled, y_train)
        tuned_models.append(('logreg', grid.best_estimator_))
    elif name == 'RandomForest':
        param_grid = {'n_estimators': [100, 200], 'max_depth': [None, 10]}
        grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3).fit(X_train_scaled, y_train)
        tuned_models.append(('rf', grid.best_estimator_))
    elif name == 'KNN':
        param_grid = {'n_neighbors': [3, 5, 7]}
        grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3).fit(X_train_scaled, y_train)
        tuned_models.append(('knn', grid.best_estimator_))
    elif name == 'GradientBoosting':
        param_grid = {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.05]}
        grid = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, cv=3).fit(X_train_scaled, y_train)
        tuned_models.append(('gb', grid.best_estimator_))
    elif name == 'DecisionTree':
        param_grid = {'max_depth': [None, 5, 10]}
        grid = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=3).fit(X_train_scaled, y_train)
        tuned_models.append(('dt', grid.best_estimator_))

# 앙상블 생성
voting_clf = VotingClassifier(estimators=tuned_models, voting='soft')
voting_clf.fit(X_train_scaled, y_train)

## 5. 최종 평가

In [ ]:
y_pred = voting_clf.predict(X_test_scaled)
print("Best 4 Ensemble Accuracy:", accuracy_score(y_test, y_pred))
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

# Confusion Matrix 시각화
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()